In [1]:
import json
import pandas as pd
import time
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import os

In [6]:
trait_name = 'attractive'
trial_csv = './'+trait_name+'/trialdata.csv'
column_names = ["subId", "trialNum", "trialId", "jsonStr"]
trial = pd.read_csv(trial_csv, names=column_names, header=None)
trial.head()

,subId,trialNum,trialId,jsonStr
0,debugGEQMFL:debugSY4EBY,0,1547796015242,"{""rt"": 75735, ""trial_type"": ""instructions"", ""v..."
1,debugGEQMFL:debugSY4EBY,1,1547796070368,"{""rt"": 55124, ""trial_type"": ""face-likert-amand..."
2,debugGEQMFL:debugSY4EBY,2,1547796074058,"{""rt"": 3688, ""trial_type"": ""face-likert-amanda..."
3,debugGEQMFL:debugSY4EBY,3,1547796077351,"{""rt"": 3291, ""trial_type"": ""face-likert-amanda..."
4,debugGEQMFL:debugSY4EBY,4,1547796079549,"{""rt"": 2195, ""trial_type"": ""face-likert-amanda..."


In [10]:
import os
likert_data_df = pd.DataFrame(columns=['subId', 'rt', 'imgName', 'rating', 'trial_index', 'debug_mode'])
likert_counter = 0

for index, row in trial.iterrows():
    json_dict = json.loads(row['jsonStr'])
    if 'task_type' in json_dict:
        if json_dict['task_type'] == 'face trials':
            likert_data_df.loc[likert_counter, 'subId'] = row['subId']
            likert_data_df.loc[likert_counter, 'rt'] = json_dict['rt']
            likert_data_df.loc[likert_counter, 'imgName'] = os.path.basename(json_dict['imgName'])
            
            likert_data_df.loc[likert_counter, 'trial_index'] = json_dict['trial_index']
            # rating encoding starts from 0, add 1 to go back to 1-9 space
            likert_data_df.loc[likert_counter, 'rating'] = json.loads(json_dict['responses'])['Q0'] + 1
            likert_data_df.loc[likert_counter, 'debug_mode'] = json_dict['debug_mode']
            likert_counter += 1
        
likert_data_df.to_csv('./'+trait_name+'/likert_data.csv')

In [12]:
likert_data_df.shape

(120, 6)

In [ ]:
# make a list for subjects
sub_num_dict = {}
sub_counter = 1
for sub_id in likert_data_df['subId']:
    if sub_id not in sub_num_dict:
        sub_num_dict[sub_id] = sub_counter
        sub_counter += 1

likert_data_df['subNum'] = likert_data_df['subId'].map(sub_num_dict)
